In [1]:
import csv
import pandas as pd
#import numpy as np
import nltk 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk import word_tokenize
import string
from nltk.tokenize import RegexpTokenizer
from itertools import chain
import json

In [5]:
#read csv as data frame with pandas
df = pd.read_csv("Airbnb_Texas_Rentals.csv")

#remove \\n new line from the description
df = df.replace(r'\\n', " ", regex = True)

#make the df as list
df1 = df.values.tolist()

#create a tsv file with \t tab space for each row 
for row in df1:
    with open("doc_tsv/doc_" +str(df1.index(row))+ ".tsv", 'w', newline='',encoding='utf8') as f_output:
        tsv_output = csv.writer(f_output, delimiter='\t',)
        tsv_output.writerow(row[1:])

In [8]:
#remove the index
for i in df1:
    i.remove(i[0])

In [9]:
#download the stop words
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jad\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
stopwords = stopwords.words('english') # get english stop words



In [11]:
#we tokenize except for the symbol $ that remains with the value
tokenizer = RegexpTokenizer("[\w'\$]+")


In [12]:
#stemming
st = PorterStemmer()

In [13]:
#tokenize description
df.description = df.description.apply(str).apply(tokenizer.tokenize)


In [14]:
#tokenize title
df.title = df.title.apply(str).apply(tokenizer.tokenize)


In [15]:
#remove stop word and punctuation from description and title
df.description = df.description.apply(lambda x : [item for item in x if item not in stopwords])

In [16]:
df.title = df.title.apply(lambda x : [item for item in x if item not in stopwords])

In [17]:
#remove stemmings from descr and title
df.description = df.description.apply(lambda x : [st.stem(y) for y in x])
 

In [18]:
df.title = df.title.apply(lambda x : [st.stem(y) for y in x])

# 3.1 Conjunctive query

### 3.1.1 Create your index!

In [19]:
voc = []

for i in list(df.description):
    for j in i:
        voc.append(j)
for i in list(df.title):
    for j in i:
        voc.append(j)
        
voc = set(voc) #all distinct words


In [20]:
num = list(voc)
vocabulary = {}

for i in num:
    vocabulary.update({num.index(i):i})

In [244]:
#vocabulary

In [21]:
with open("vocabulary.json", "w", encoding = "utf8") as myfile:
    myfile.write(json.dumps(vocabulary))

In [22]:
#create dict from df that assign for each rows all the words in
newdict = {}
for index, row in df.iterrows():
    newdict["doc_" + str(index)] = list(row["description"]) + list(row["title"])

In [231]:
newdict

{'doc_0': ['welcom',
  'stay',
  'privat',
  'room',
  'queen',
  'bed',
  'detach',
  'privat',
  'bathroom',
  'second',
  'floor',
  'anoth',
  'privat',
  'bedroom',
  'sofa',
  'bed',
  'avail',
  'addit',
  'guest',
  '10$',
  'addit',
  'guest',
  '10min',
  'iah',
  'airport',
  'airport',
  'pick',
  'drop',
  'avail',
  '$10',
  'trip',
  '2',
  'privat',
  'room',
  'bathroom',
  '10min',
  'iah',
  'airport'],
 'doc_1': ['stylish',
  'fulli',
  'remodel',
  'home',
  'upscal',
  'NW',
  'alamo',
  'height',
  'area',
  'amaz',
  'locat',
  'hous',
  'conveni',
  'locat',
  'quiet',
  'street',
  'beauti',
  'season',
  'tree',
  'prestigi',
  'neighborhood',
  'close',
  'airport',
  '281',
  '410',
  'loop',
  'town',
  'area',
  'featur',
  'open',
  'floor',
  'plan',
  'origin',
  'hardwood',
  'floor',
  '3',
  'bedroom',
  '3',
  'full',
  'bathroom',
  'independ',
  'garden',
  'TV',
  'room',
  'sleep',
  '2',
  'european',
  'inspir',
  'kitchen',
  'top',
  'line'

In [23]:
#create inverted index called inverse

inverse = dict() 
for key in newdict: 
    # Go through the list that is saved in the dict:
    for item in newdict[key]:
        # Check if in the inverted dict the key exists
        if item not in inverse: 
            # If not create a new list
            inverse[item] = [key] 
        else: 
            inverse[item].append(key) 


In [27]:
# change the keys of inverse with the keys of vocabulary
for i in vocabulary:
    inverse[i] = inverse[vocabulary[i]]
    del inverse[vocabulary[i]]

In [32]:
#save inverted index as json
with open("inverted_index.json", "w", encoding = "utf8") as myfile:
    myfile.write(json.dumps(inverse))

In [116]:
#create query
with open('vocabulary.json') as F:
    vocabulary = json.load(F)
vocabulary={k:v for v,k in vocabulary.items()}
with open('inverted_index.json') as F:
    index = json.load(F)
query=input()
query=tokenizer.tokenize(query)
query=[t for t in query if t not in stopwords]
query=[st.stem(word) for word in query]
index = json.loads(open('inverted_index.json').read())
vocab = json.loads(open('vocabulary.json').read())
for i in set(query):
    if i in vocabulary:
        query.remove(i)
        query.append(vocabulary[i])
    else:
        query.remove(i)
docs=[]
for i in query:
    docs.append(set(index[i]))
docs=set.intersection(*docs)
results=[]
for file in docs:
    opentsv=open("doc_tsv/"+file+".tsv","r",encoding="utf8")
    lines=opentsv.readlines()
    for x in lines:
        results.append([x.split('\t')[7],x.split('\t')[4],x.split('\t')[2],x.split('\t')[8]])
    opentsv.close()
result=pd.DataFrame(results,columns=['Title','Description','City','URL'])
def make_clickable(val):
    # target _blank to open new window
    return '<a target="_blank" href="{}">{}</a>'.format(val, val)
result.style.set_properties(**{'text-align': 'center'})
result.style.format({'URL': make_clickable})

dog cat beach


,Title,Description,City,URL
0,Maui East-2BR-1.5 BA-Walk to Beach!,"My place is close to restaurants and dining, beach, shopping, Hwy 361. You’ll love my place because of the location, the outdoors space, the neighborhood, close walk to beach. Large pool. My place is good for couples, families (with kids), and small dogs (no more than two). Keep dog poundage under 45# total. No cats, please. City of Port Aransas Reg.#382633",Port Aransas,https://www.airbnb.com/rooms/13409801?location=Corpus%20Christi%2C%20TX
1,Maui East-2BR-1.5 BA-Walk to Beach!,"My place is close to restaurants and dining, beach, shopping, Hwy 361. You’ll love my place because of the location, the outdoors space, the neighborhood, close walk to beach. Large pool. My place is good for couples, families (with kids), and small dogs (no more than two). Keep dog poundage under 45# total. No cats, please. City of Port Aransas Reg.#382633",Port Aransas,https://www.airbnb.com/rooms/13409801?location=Aransas%20Pass%2C%20TX
